In [1]:
'''Script to generate poetry from Yeats.'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import re

Using TensorFlow backend.


In [3]:
text = open('yeats.txt').read().lower()
all_words = re.findall(r"[\w']+|[.,!?;]", text)

words = sorted(list(set(all_words)))

print('corpus length:', len(all_words))
print('total unique words:', len(words))

word_indices = dict((w, i) for i, w in enumerate(words))
indices_word = dict((i, w) for i, w in enumerate(words))

# cut the text in semi-redundant sequences of maxlen words
maxlen = 20
step = 1
sentences = []
next_words = []
for i in range(0, len(all_words) - maxlen, step):
    sentences.append(all_words[i: i + maxlen])
    next_words.append(all_words[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)
y = np.zeros((len(sentences), len(words)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        X[i, t, word_indices[word]] = 1
    y[i, word_indices[next_words[i]]] = 1

corpus length: 29613
total unique words: 3586
nb sequences: 29593
Vectorization...


In [4]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(words))))
model.add(Dense(len(words)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [5]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [6]:
# train the model, output generated text after each iteration
for iteration in range(1, 30):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(all_words) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        #generated = ' '
        sentence = all_words[start_index: start_index + maxlen]
        #generated.join(sentence)
        
        sentence_str = ' '.join(sentence)
        print('----- Generating with seed: "' , sentence_str , '"')
        sys.stdout.write(sentence_str)
        sys.stdout.write(' ')

        for i in range(120):
            x = np.zeros((1, maxlen, len(words)))
            for t, word in enumerate(sentence):
                x[0, t, word_indices[word]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            #generated += next_word
            del sentence[0]
            sentence.append(next_word)
            #print(sentence)
            #print(next_word)

            if next_word != '.' and next_word != '!' and next_word != '?' and next_word != ',' and next_word != ';':
                sys.stdout.write(' ')
            sys.stdout.write(next_word)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Epoch 1/1
29593/29593 [==============================] - 95s - loss: 6.1030    

----- diversity: 0.2
----- Generating with seed: " live on when all this heavy history s done ; our hands , our hands must ebb away . _three "
live on when all this heavy history s done ; our hands , our hands must ebb away . _three  in the world, and the world of the world. the world of the world, and the sea of the world. the world of the world, and the world of the world. the world of the world. the world of the sea, and the world, and the sea, and the world of the world, and the world of the world, and the world of the stars. the world of the sea, and the world of the world, and the world of the sea. and the world of the world, and the world, and the world and the world, and the world of

----- diversity: 0.5
----- Generating with seed: " live on when all this heavy history s done ; our hands , our hands must ebb away . _three "
live on wh

In [8]:
from keras.models import load_model
model.save('yeats.h5')

In [25]:
diversity = .7
for j in xrange(10):
    start_index = random.randint(0, len(all_words) - maxlen - 1)
    sentence = all_words[start_index: start_index + maxlen]

    sentence_str = ' '.join(sentence)
    print()
    print('----- Generating -----')
    #sys.stdout.write(sentence_str)
    #sys.stdout.write(' ')

    for i in range(120):
        x = np.zeros((1, maxlen, len(words)))
        for t, word in enumerate(sentence):
            x[0, t, word_indices[word]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_word = indices_word[next_index]

        del sentence[0]
        sentence.append(next_word)

        #if next_word != '.' and next_word != '!' and next_word != '?' and next_word != ',' and next_word != ';':
        #    sys.stdout.write(' ')
        sys.stdout.write(next_word)
        sys.stdout.write(' ')
        sys.stdout.flush()
        #print()


----- Generating -----
they had come to the happy . and burning , weep for the stars were ancient night west , and the days when away from the isle , woven away , in music of golden beauty . a she s heavy heart , the pale glimmering over the thing . and woods were gone , and dream i love , and kiss her songs . go , always my deer , mind ; and the without its their us , to the stars and merry night . and , as a grass starry pass ; but the sound of the blue star branch there s no dim sound ; and only hearts are dream her dream ? o , mortal 
----- Generating -----
she life a high heart , as though they have come to his care ; he set coming of robes of the dream of love died , and all their them to sleep for love the oisin s a their hearts were well ; they bid years little way into the forest , but down to your one comes in her hair , the lips that lay peace in soft red a mind , when all man a sleep sound in the times , to hill where the sword care i are away in the snow of human , and th